In [103]:
import requests
import pandas as pd
import numpy as np
import zipfile
import io
import matplotlib.pyplot as plt
import glob



In [102]:

import requests, zipfile, io
r = requests.get("https://ies-python-midterm.s3.eu-central-1.amazonaws.com/studentsSets/justynaCase.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("/Users/jirka_12/Documents/4.semestr/DP_python/data")




In [117]:
csv_1 = pd.read_csv("/Users/jirka_12/Documents/4.semestr/DP_python/data/AAPL.csv")
csv_1["Company"] = "AAPL"
csv_2 = pd.read_csv("/Users/jirka_12/Documents/4.semestr/DP_python/data/ABBV.csv")
csv_2["Company"] = "ABBV"
csv_3 = pd.read_csv("/Users/jirka_12/Documents/4.semestr/DP_python/data/ABC.csv")
csv_3["Company"] = "ABC"
csv_4 = pd.read_csv("/Users/jirka_12/Documents/4.semestr/DP_python/data/ABMD.csv")
csv_4["Company"] = "ABMD"
data = pd.concat([csv_1,csv_2,csv_4,csv_3])
data


,Date,High,Low,Open,Close,Volume,Adj Close,Company
0,2017-11-13,43.625000,43.349998,43.375000,43.492500,67928400.0,41.357002,AAPL
1,2017-11-14,43.369999,42.794998,43.259998,42.834999,99130000.0,40.731789,AAPL
2,2017-11-15,42.580002,42.095001,42.492500,42.270000,116632400.0,40.194534,AAPL
3,2017-11-16,42.967499,42.575001,42.794998,42.775002,94550000.0,40.674740,AAPL
4,2017-11-17,42.847500,42.410000,42.759998,42.537498,87598000.0,40.448902,AAPL
...,...,...,...,...,...,...,...,...
1254,2022-11-07,163.610001,160.550003,161.940002,162.009995,1393600.0,161.513275,ABC
1255,2022-11-08,160.149994,154.910004,154.910004,159.660004,8144900.0,159.170502,ABC
1256,2022-11-09,162.389999,157.809998,159.660004,158.190002,1980700.0,157.705002,ABC
1257,2022-11-10,161.339996,153.779999,161.199997,155.000000,4479300.0,155.000000,ABC


In [118]:
grouped = data.groupby('Company')
for name, group in grouped:
    if (group['Close'] == group['Adj Close']).all():
        print(name)


ABMD


In [119]:
grouped = data.groupby('Company')
for name, group in grouped:
    highest_price = group['Adj Close'].max()
    lowest_price = group['Adj Close'].min()
    print(f"{name}: highest price = {highest_price}, lowest price = {lowest_price}")

AAPL: highest price = 180.9597320556641, lowest price = 34.30958938598633
ABBV: highest price = 170.14651489257812, lowest price = 54.02183151245117
ABC: highest price = 164.20936584472656, lowest price = 66.3444595336914
ABMD: highest price = 449.75, lowest price = 130.50999450683594


In [120]:
grouped = data.groupby('Company')
total_returns = {}
for name, group in grouped:
    start_price = group.iloc[0]['Adj Close']
    end_price = group.iloc[-1]['Adj Close']
    total_return = (end_price / start_price - 1) * 100
    total_returns[name] = total_return
max_total_return = max(total_returns, key=total_returns.get)
print(f"The company with the highest total return is {max_total_return} with a total return of {total_returns[max_total_return]:.2f}%")

The company with the highest total return is AAPL with a total return of 261.97%


Problem 2


Loading data

In [82]:
df = pd.read_csv("https://ies-python-midterm.s3.eu-central-1.amazonaws.com/companies/companies_no_subindustry.csv")
df


,Symbol,Security,GICS Sector,hq,included,Founded
0,MMM,3M,Industrials,"Saint Paul, Minnesota",09.08.1976,1902
1,AOS,A. O. Smith,Industrials,"Milwaukee, Wisconsin",26.07.2017,1916
2,ABT,Abbott,Health Care,"North Chicago, Illinois",31.03.1964,1888
3,ABBV,AbbVie,Health Care,"North Chicago, Illinois",31.12.2012,2013 (1888)
4,ABMD,Abiomed,Health Care,"Danvers, Massachusetts",31.05.2018,1981
...,...,...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,"Louisville, Kentucky",06.10.1997,1997
499,ZBRA,Zebra Technologies,Information Technology,"Lincolnshire, Illinois",23.12.2019,1969
500,ZBH,Zimmer Biomet,Health Care,"Warsaw, Indiana",07.08.2001,1927
501,ZION,Zions Bancorporation,Financials,"Salt Lake City, Utah",22.06.2001,1873


Calculating how many company did not filled in in collumn included

In [45]:
num_missing = df['included'].isna().sum()
print(f"Number of companies that did not filled in {num_missing}")


Number of companies that did not filled in 45


Delete the companies with no inclusion date and calculate which company is the oldest/youngest constituent and tell us the average age of a constituent in the sample.

In [55]:
df = df.dropna(subset=["included"])
df["included"] = pd.to_datetime(df["included"], dayfirst=True, errors='coerce')
df["age"] = (pd.to_datetime("today") - df["included"]).astype('<m8[Y]')
oldest = df.loc[df["age"].idxmax()]
youngest = df.loc[df["age"].idxmin()]
average_age = np.mean(df["age"])

print(f"The oldest constituent is {oldest}")
print(f"The youngest constituent is {youngest}")
print(f"The average age of constituent is {average_age}")


The oldest constituent is Symbol                          MO
Security                    Altria
GICS Sector       Consumer Staples
hq              Richmond, Virginia
included       1957-03-04 00:00:00
Founded                       1985
age                           66.0
Name: 25, dtype: object
The youngest constituent is Symbol                        ACGL
Security        Arch Capital Group
GICS Sector             Financials
hq               Hamilton, Bermuda
included       2022-11-01 00:00:00
Founded                       1995
age                            0.0
Name: 48, dtype: object
The average age of constituent is 21.26477024070022


Describe the distribution of companies across sectors and create a plot that
demonstrates the proportionality of the sectors (i.e. pie plot, or something like this)

In [100]:
df.columns

Index(['Symbol', 'Security', 'GICS Sector', 'hq', 'included', 'Founded',
       'State'],
      dtype='object')

In [99]:

sector_counts = df['GICS Sector'].value_counts()




KeyError: 'GICS Sector'

Parse the “hq” column, extract the state of the hq and describe the distribution of the
states

In [90]:
df['State'] = df['hq'].str.split(', ').str[-2]
state_counts = df['State'].value_counts()
state_counts


New York City    43
Houston          19
Chicago          15
Atlanta          14
Dallas           10
                 ..
Winona            1
Akron             1
Brookfield        1
Dearborn          1
Parsippany        1
Name: State, Length: 238, dtype: int64

Joing data

In [77]:
df_2 = pd.read_csv("https://ies-python-midterm.s3.eu-central-1.amazonaws.com/companies/companies_subindustry.csv")
merged_df = pd.merge(df,df_2, on='Symbol')

consumer_df = merged_df[merged_df['GICS Sector'] == 'Consumer Discretionary']

subindustry_counts = consumer_df['GICS Sub-Industry'].value_counts()

subindustry_counts.plot(kind='bar', rot=90)
plt.title('Distribution of Sub-Industries in Consumer Discretionary Sector')
plt.xlabel('Sub-Industry')
plt.ylabel('Count')
plt.show()


KeyError: 'GICS Sector'

In [ ]:
plt.pie(sector_counts.values, labels=sector_counts.index, autopct='%1.1f%%')
plt.title('Proportion of Companies in Each Sector')
plt.show()